In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [2]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [3]:
string = """F10
N3
F7
R90
F11"""

In [4]:
def parse_input(input: str):
    dir, val = re.search(r'(\w)(\d+)', input).groups()
    return dir, int(val)

In [5]:
test_ins = read_data(string, parser=parse_input, sep="\n", testing=True)
test_ins


[('F', 10), ('N', 3), ('F', 7), ('R', 90), ('F', 11)]

Part I  

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [6]:
angles = dict(
    E=90,
    S=180,
    W=270,
    N=0
)
directions = dict([(value, key) for key, value in angles.items()])

def run_part1(moves: Tuple[str, int], start_dir='E'):
    distances = Counter()
    current_dir = start_dir
    current_angle = angles[current_dir]
    for ins, val in moves:
        if ins in 'F':
            distances[current_dir] += val
        elif ins in 'L':
            current_angle = (current_angle - val) % 360
        elif ins in 'R':
            current_angle = (current_angle + val) % 360 
        else:
            distances[ins] += val
        current_dir = directions[current_angle]
    return sum(map(abs, (distances['N'] - distances['S'], distances['E'] - distances['W'])))


In [7]:
run_part1(test_ins)

25

In [8]:
real_ins = read_data("input.txt", parser=parse_input)
run_part1(real_ins)

364

Part II

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [9]:
def update_dir(curr_dir: str, val: int, mov: int) -> str:
    # postive val means clockwise, negative means counterclockwise
    angle = angles[curr_dir]
    new_angle = (angle + val) % 360
    new_dir = directions[new_angle]
    return new_dir, mov

def run_part2(moves: Tuple[str, int], start_wp={'N':1, 'S':0, 'E':10, 'W':0}):
    distances = Counter()
    current_wp = start_wp
    for ins, val in moves:
        if ins in 'F':
            for direct, mov in current_wp.items():
                distances[direct] += val * mov
        elif ins in 'L':
            current_wp = dict([update_dir(direct, -val, mov) for direct, mov in current_wp.items()])
        elif ins in 'R':
            current_wp = dict([update_dir(direct, val, mov) for direct, mov in current_wp.items()])
        else:
            current_wp[ins] += val
    return sum(map(abs, (distances['N'] - distances['S'], distances['E'] - distances['W'])))

In [10]:
run_part2(test_ins, start_wp={'N':1, 'S':0, 'E':10, 'W':0})

286

In [11]:
run_part2(real_ins, start_wp={'N':1, 'S':0, 'E':10, 'W':0})

39518